# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df=pd.DataFrame(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

    Antes de ajustar o modelo de log(renda) predito por todas as variáveis, vamos encontrar as categorias mais frequêntes das nossas variáveis qualitativas. As que tem apenas duas opções, como sexo, vamos deixar, agora as que possuem mais opções, vamos encontrar qual valor que mais aparece para assim utilizar como casela. Para ter certeza que selecionamos a casela correta, vamos exibir as matrizes antes de construir o modelo de regressão com todas as variáveis juntas. 

In [5]:
#1) vamos analisar qual valor mais recorrente na variável tipo_renda. 
df.tipo_renda.value_counts()

Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64

In [6]:
#agora vamos observar a matriz para ver se assalariado está como casela ou precisaremos mudar. 
y, x = patsy.dmatrices('np.log(renda) ~ C(tipo_renda)', data = df)
x

DesignMatrix with shape (15000, 5)
  Columns:
    ['Intercept',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]']
  Terms:
    'Intercept' (column 0), 'C(tipo_renda)' (columns 1:5)
  (to view full data, use np.asarray(this_obj))

    A primeira variável pode ser inserida no modelo apenas utilizando o C para categorizar, pois já tem como casela assalariado.
    Agora vamos analisar a segunda variável:

In [10]:
#vamos analisar qual valor mais recorrente na variável educação
df.educacao.value_counts()

Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64

In [11]:
#agora vamos observar e ajustar a matriz para que secundário seja nossa casela.
y, x = patsy.dmatrices('np.log(renda) ~ C(educacao, Treatment(2))', data = df)
x

DesignMatrix with shape (15000, 5)
  Columns:
    ['Intercept',
     'C(educacao, Treatment(2))[T.Primário]',
     'C(educacao, Treatment(2))[T.Pós graduação]',
     'C(educacao, Treatment(2))[T.Superior completo]',
     'C(educacao, Treatment(2))[T.Superior incompleto]']
  Terms:
    'Intercept' (column 0), 'C(educacao, Treatment(2))' (columns 1:5)
  (to view full data, use np.asarray(this_obj))

    Na categorização da nossa variável educação, precisou-se utilizar um tratamento para que a casela se tornasse Secundário, portanto será inserida no nosso modelo com o Treatment. Agora vamos analisar tipo_residencia:

In [13]:
#vamos analisar qual valor mais recorrente na variável tipo_residencia:
df.tipo_residencia.value_counts()

Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: tipo_residencia, dtype: int64

In [14]:
#agora vamos observar a matriz para ver se casa está como casela ou precisaremos mudar. 
y, x = patsy.dmatrices('np.log(renda) ~ C(tipo_residencia, Treatment(1))', data = df)
x

DesignMatrix with shape (15000, 6)
  Columns:
    ['Intercept',
     'C(tipo_residencia, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia, Treatment(1))[T.Comunitário]',
     'C(tipo_residencia, Treatment(1))[T.Estúdio]',
     'C(tipo_residencia, Treatment(1))[T.Governamental]']
  Terms:
    'Intercept' (column 0)
    'C(tipo_residencia, Treatment(1))' (columns 1:6)
  (to view full data, use np.asarray(this_obj))

    Acima também precisou-se utilizar um Treatment, a fim de deixar casa, a variável mais frequente, como casela de referência. Por fim, analisaremos estado_civil:

In [15]:
#vamos analisar qual valor mais recorrente na variável estado_civil:
df.estado_civil.value_counts()

Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64

In [16]:
#Observando a classificação feita pela matriz
y, x = patsy.dmatrices('np.log(renda) ~ C(estado_civil, Treatment(0))', data = df)
x

DesignMatrix with shape (15000, 5)
  Columns:
    ['Intercept',
     'C(estado_civil, Treatment(0))[T.Separado]',
     'C(estado_civil, Treatment(0))[T.Solteiro]',
     'C(estado_civil, Treatment(0))[T.União]',
     'C(estado_civil, Treatment(0))[T.Viúvo]']
  Terms:
    'Intercept' (column 0)
    'C(estado_civil, Treatment(0))' (columns 1:5)
  (to view full data, use np.asarray(this_obj))

Acima, como a classificação natural não deixou casado como casela, nos o colocamos utilizando o Treatment, logo, estado_civil irá para o modelo de regressão com o Treatment.

In [17]:
#1) regressão: log(renda) explicado por todas as demais, incluindo as categorias e seus devidos tratamentos. 
reg1=smf.ols('np.log(renda)~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        09:20:29   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5264      0.219     29.853      0.000       6.098       6.955
sexo[T.M]                                             0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                              0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                               0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda)[T.Bolsista]                             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]                           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]                         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]                     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(0))[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(0))[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(0))[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

Nesse modelo temos um R-quadrado de 35,7 e um R-ajustado de 35,6. Temos p-values bem superiores a 5% e inicialmente iremos retirar a variável com menor significância apontada pelo p-value, é ela tipo de residência.

In [19]:
#2) regressão: log de renda explicada por todas as demais, exceto tipo_residência:
reg1=smf.ols('np.log(renda)~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil, Treatment(0)) + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        09:24:13   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.5315      0.218     29.894      0.000       6.103       6.960
sexo[T.M]                                            0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                             0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                              0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda)[T.Bolsista]                            0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda)[T.Empresário]                          0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda)[T.Pensionista]                        -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda)[T.Servidor público]                    0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment(2))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment(2))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]            0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment(0))[T.Solteiro]            0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment(0))[T.União]              -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment(0))[T.Viúvo]               0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.13e+

Manteve-se o R-quadrado e também o R-ajustado, portanto, fizemos uma boa exclusão. Agora, de acordo com o que é solicitado no exercício 3, iremos prosseguir com a análise e ir retirando as variáveis menos significantes.

In [26]:
#3) regressão: log de renda explicada por todas as demais, exceto tipo_residência e estado civil (observado no modelo acima):
reg1=smf.ols('np.log(renda)~ sexo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     490.0
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        09:44:39   Log-Likelihood:                -13582.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12412   BIC:                         2.731e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.1099      0.045    157.698      0.000       7.021       7.198
sexo[T.M]                                            0.8014      0.014     58.023      0.000       0.774       0.828
posse_de_imovel[T.True]                              0.0830      0.014      6.050      0.000       0.056       0.110
C(tipo_renda)[T.Bolsista]                            0.2035      0.241      0.843      0.399      -0.269       0.677
C(tipo_renda)[T.Empresário]                          0.1538      0.015     10.318      0.000       0.125       0.183
C(tipo_renda)[T.Pensionista]                        -0.3280      0.241     -1.360      0.174      -0.801       0.145
C(tipo_renda)[T.Servidor público]                    0.0585      0.022      2.636      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0158      0.072      0.220      0.826      -0.125       0.157
C(educacao, Treatment(2))[T.Pós graduação]           0.1266      0.142      0.891      0.373      -0.152       0.405
C(educacao, Treatment(2))[T.Superior completo]       0.1110      0.014      8.024      0.000       0.084       0.138
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0310      0.032     -0.961      0.337      -0.094       0.032
qtd_filhos                                           0.0269      0.019      1.392      0.164      -0.011       0.065
idade                                                0.0056      0.001      7.162      0.000       0.004       0.007
tempo_emprego                                        0.0615      0.001     59.336      0.000       0.059       0.064
qt_pessoas_residencia                                0.0020      0.016      0.126      0.899      -0.030       0.034
==============================================================================
Omnibus:                        0.670   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.715   Jarque-Bera (JB):                0.652
Skew:                           0.017   Prob(JB):                        0.722
Kurtosis:                       3.012   Cond. No.                     1.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

    Com a retirada da variável estado_civil temos uma queda tanto no R-quadrado, tanto no ajustado, mas é normal que ao diminuir o número de variáveis explicativas, diminua também o R-quadrado, mas foi uma queda mínima. Também houve um aumento significativo no p-value de qt_pessoas_residência, indicando que agora essa variável já não tem mais nenhuma significância e por isso será a próxima a ser excluída. 

In [31]:
#3) regressão: log de renda explicada por sexo, posse_de_imovel, qtd_filhos, tipo_renda, educação, idade e tempo_emprego
reg1=smf.ols('np.log(renda)~ sexo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + idade + tempo_emprego', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     527.7
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        10:00:31   Log-Likelihood:                -13582.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12413   BIC:                         2.730e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.1134      0.036    200.373      0.000       7.044       7.183
sexo[T.M]                                            0.8016      0.014     58.362      0.000       0.775       0.829
posse_de_imovel[T.True]                              0.0830      0.014      6.050      0.000       0.056       0.110
C(tipo_renda)[T.Bolsista]                            0.2041      0.241      0.846      0.398      -0.269       0.677
C(tipo_renda)[T.Empresário]                          0.1537      0.015     10.318      0.000       0.125       0.183
C(tipo_renda)[T.Pensionista]                        -0.3292      0.241     -1.366      0.172      -0.802       0.143
C(tipo_renda)[T.Servidor público]                    0.0585      0.022      2.636      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0158      0.072      0.220      0.826      -0.125       0.156
C(educacao, Treatment(2))[T.Pós graduação]           0.1265      0.142      0.890      0.374      -0.152       0.405
C(educacao, Treatment(2))[T.Superior completo]       0.1110      0.014      8.023      0.000       0.084       0.138
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0310      0.032     -0.962      0.336      -0.094       0.032
qtd_filhos                                           0.0290      0.009      3.381      0.001       0.012       0.046
idade                                                0.0056      0.001      7.163      0.000       0.004       0.007
tempo_emprego                                        0.0615      0.001     59.341      0.000       0.059       0.064
==============================================================================
Omnibus:                        0.671   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.715   Jarque-Bera (JB):                0.652
Skew:                           0.017   Prob(JB):                        0.722
Kurtosis:                       3.012   Cond. No.                     1.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

    Manteve-se o R-quadrado, o que nos mostra que a escolha da variável a ser removida foi bem sucedida. Com o novo ajuste do modelo percebemos que a variável de menos significância é agora educação.

In [32]:
#3) regressão: log de renda explicada por sexo, posse_de_imovel, qtd_filhos, tipo_renda, idade e tempo_emprego
reg1=smf.ols('np.log(renda)~ sexo + posse_de_imovel + qtd_filhos + C(tipo_renda) + idade + tempo_emprego', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     750.3
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        10:02:31   Log-Likelihood:                -13618.
No. Observations:               12427   AIC:                         2.726e+04
Df Residuals:                   12417   BIC:                         2.733e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             7.1654      0.034    210.573      0.000       7.099       7.232
sexo[T.M]                             0.7970      0.014     57.930      0.000       0.770       0.824
posse_de_imovel[T.True]               0.0863      0.014      6.284      0.000       0.059       0.113
C(tipo_renda)[T.Bolsista]             0.2813      0.242      1.164      0.245      -0.193       0.755
C(tipo_renda)[T.Empresário]           0.1642      0.015     11.038      0.000       0.135       0.193
C(tipo_renda)[T.Pensionista]         -0.2684      0.242     -1.111      0.267      -0.742       0.205
C(tipo_renda)[T.Servidor público]     0.0770      0.022      3.475      0.001       0.034       0.120
qtd_filhos                            0.0304      0.009      3.529      0.000       0.014       0.047
idade                                 0.0051      0.001      6.700      0.000       0.004       0.007
tempo_emprego                         0.0616      0.001     59.246      0.000       0.060       0.064
==============================================================================
Omnibus:                        0.992   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.609   Jarque-Bera (JB):                0.963
Skew:                           0.019   Prob(JB):                        0.618
Kurtosis:                       3.021   Cond. No.                     1.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

    Conforme mencionado anteriormente, ao remover variáveis explicativas é normal que o R-quadrado diminua, houve novamente uma diiminuição, porém desde o ínicio das nossas remoções ainda não perdeu-se nem 1% do R-quadrado e já foram retiradas 3 variáves, portanto, estamos tornando nosso modelo mais simples sem perder qualidade de explicação. A próxima variável a ser removida será tipo_renda.

In [33]:
#3) regressão: log de renda explicada por sexo, posse_de_imovel, qtd_filhos, idade e tempo_emprego
reg1=smf.ols('np.log(renda)~ sexo + posse_de_imovel + qtd_filhos + idade + tempo_emprego', data = df).fit()

reg1.summary() #resumo da regressão

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     1312.
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        10:06:00   Log-Likelihood:                -13681.
No. Observations:               12427   AIC:                         2.737e+04
Df Residuals:                   12421   BIC:                         2.742e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.2434      0.033    216.813      0.000       7.178       7.309
sexo[T.M]                   0.7877      0.014     57.104      0.000       0.761       0.815
posse_de_imovel[T.True]     0.0883      0.014      6.400      0.000       0.061       0.115
qtd_filhos                  0.0255      0.009      2.951      0.003       0.009       0.042
idade                       0.0048      0.001      6.224      0.000       0.003       0.006
tempo_emprego               0.0609      0.001     58.994      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.184   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.553   Jarque-Bera (JB):                1.163
Skew:                           0.023   Prob(JB):                        0.559
Kurtosis:                       3.013   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

    Acredito que agora temos nosso modelo final. Ele tem um R-quadrado de 1,1% menor que o do primeiro modelo, porém ele tem 5 variáveis a menos que o primeiro, sendo portanto, muito mais simples que aquele e consequentemente, mais eficaz e sem risco de overffiting como era o caso daquele. Agora temos apenas variáveis significantes e um R-ajustado quase igual ao R-quadrado. 